In [20]:
import sys
sys.path.insert(1, '../src')
from imports import *
from dataset_utilities import *
from model_aplication import *

Preprocessing necessari per poder fer la execucciò

In [21]:
dataset = load_dataset('..\data\smart_grid_stability_augmented.csv')
label_encoder = sklearn.preprocessing.LabelEncoder()
dataset['stabf']= label_encoder.fit_transform(dataset['stabf'])
data = dataset.values
y = data[:,-2]
y_bin = data[:,-1].astype(int)
x = data[:, :12]
x_train, y_train, x_val, y_val = split_data(x, y_bin)
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l2', solver='liblinear'))
sel_.fit(x_train, np.ravel(y_train, order='C'))
sel_.get_support()
x_train = pd.DataFrame(x_train)
selected_feat = x_train.columns[(sel_.get_support())]
x_train_selected = sel_.transform(x_train)
x_test_selected = sel_.transform(x_val)
x_train_selected.shape, x_train_selected.shape
x_selected = np.concatenate((x_train_selected, x_test_selected))
y_selected = np.concatenate((y_train, y_val))
dataset_x = dataset.drop(columns='stabf')
scaler = MinMaxScaler()
data_rescaled = scaler.fit_transform(dataset_x)
pca2 = PCA(n_components = 2)
pca2.fit(data_rescaled)
reduced = pca2.transform(data_rescaled)
df2 = pd.DataFrame(reduced, columns = ['pc1','pc2'])
df2['stabf'] = dataset['stabf']
pca95 = PCA(n_components = 0.95, svd_solver='full')
pca95.fit(data_rescaled)
reduced = pca95.transform(data_rescaled)
df95 = pd.DataFrame(reduced, columns = ['pc1','pc2','pc3','pc4','pc5','pc6','pc7','pc8','pc9','pc10','pc11'])
df95['stabf']=dataset['stabf']

Apliquem models

In [22]:
particions = [0.5, 0.7, 0.8]
probs_list,x_t, x_v, y_t, y_v=apply_models(particions,x_selected,y_selected,ret=True)

Correct classification Logistic         0.5 % of the data:  0.7210333333333333


KeyboardInterrupt: 

In [9]:
X2 = df2.drop(['stabf'],axis=1)
y2 = df2.filter(['stabf'])
apply_models(particions,X2,y2)

In [ ]:
X95 = df95.drop(['stabf'],axis=1)
y95 = df95.filter(['stabf'])
apply_models(particions,X95,y95)

Observem que el millor es el model de amb un PCA de dues dimensions sobre la classificaciò feta per el KNN amb 3 veins, amb l'algoritme kd_tree i que els pesos vinguin donats per la distancia.

Comprovem amb cross-validation si les dades són certes

In [24]:
x_t, x_v, y_t, y_v = train_test_split(X2, y2, train_size=0.7)
pipeline = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=3,algorithm="kd_tree",weights="distance"))

scores = cross_val_score(pipeline, X=x_t, y=y_t, cv=1000, n_jobs=-1)
 
print('Cross Validation accuracy scores: %s' % scores)
 
print('Cross Validation accuracy: %.3f +/- %.3f' % (np.mean(scores),np.std(scores)))

Cross Validation accuracy scores: [1.         1.         1.         1.         1.         1.
 0.97619048 1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.        